In [ ]:
from operators import *
from Solutions import *
import numpy as np
import time

def p_selection(it, p_init, n_queries):
    it = int(it / n_queries * 10000)
    if 0 < it <= 50:
        p = p_init / 2
    elif 50 < it <= 200:
        p = p_init / 4
    elif 200 < it <= 500:
        p = p_init / 5
    elif 500 < it <= 1000:
        p = p_init / 6
    elif 1000 < it <= 2000:
        p = p_init / 8
    elif 2000 < it <= 4000:
        p = p_init / 10
    elif 4000 < it <= 6000:
        p = p_init / 12
    elif 6000 < it <= 8000:
        p = p_init / 15
    elif 8000 < it:
        p = p_init / 20
    else:
        p = p_init

    return p


class Population:
    def __init__(self, solutions: list, loss_function):
        self.population = solutions
        self.fronts = None
        self.loss_function = loss_function

    def evaluate(self):
        for pi in self.population:
            pi.evaluate(self.loss_function)

    def find_adv_solns(self):
        adv_solns = []
        for pi in self.population:
            if pi.is_adversarial:
                adv_solns.append(pi)

        return adv_solns


class Attack:
    def __init__(self, params):
        self.params = params
        self.fitness = []
        self.data = []

    def completion_procedure(self, population: Population, loss_function, fe, success):

            adversarial_labels = []
            for soln in population.fronts[0]:
                adversarial_labels.append(loss_function.get_label(soln.generate_image()))

            d = {"front0_imgs": [soln.generate_image() for soln in population.fronts[0]],
                "true_label": loss_function.true,
                "adversarial_labels": adversarial_labels, #predicted labels
                "front0_fitness": [soln.fitnesses for soln in population.fronts[0]],
                "success_rate": [soln.is_adversarial for soln in population.fronts[0]]
                #  "model": model.chosen_model,
                }

            title = f'results/{d["true_label"]}_{self.params["i"]}.npy'
            np.save(title, d, allow_pickle=True)

    def attack(self, loss_function):
        start = time.time()
        h, w, c = self.params["x"].shape # The test image x_test, a tensor
        pm_pixels = self.params["pm_pixels"]
        n_pixels = h * w # without considering channels
        all_pixels = np.arange(n_pixels)
        ones_prob = (1 - self.params["zero_probability"]) / 2
        init_solutions = []
        values_array = np.zeros((self.params["eps"],3))
        # print("about to create initial population")
        # start_init_pop = time.time()
        for _ in range(self.params["pop_size"]):
          pixel_count = int(np.floor(np.random.choice(self.params["init_pixels"],1)*n_pixels))
          perturbed_indices = np.random.choice(np.arange(self.params["eps"]),size=pixel_count, replace=False)
          perturbed_values = values_array.copy()
          perturbed_values[perturbed_indices] = np.random.choice([-1, 1, 0], size=(pixel_count, 3), p=(ones_prob, ones_prob, 0.3))
          # temp_solution = Solution(pixels = np.random.choice(all_pixels, size=self.params["eps"], replace=False),
          #                 values = perturbed_values,
          #                 x = self.params["x"].copy(),
          #                 p_size = np.random.choice(self.params["init_p_size"],1))
          temp_solution = Solution(pixels = all_pixels,
                                   values = perturbed_values,
                                   x = self.params["x"].copy(),
                                   p_size = np.random.choice(self.params["init_p_size"],1))
          init_solutions.append(temp_solution)

        # print("init population:",time.time()-start_init_pop)
        population = Population(init_solutions, loss_function)
        population.evaluate()
        fe = len(population.population)
        for it in range(1, self.params["iterations"]):
          print(it)
          population.fronts = fast_nondominated_sort(population.population)

          adv_solns = population.find_adv_solns()

          self.fitness.append(min(population.population, key=attrgetter('loss')).fitnesses)

          for front in population.fronts:
            calculate_crowding_distance(front)
          parents = tournament_selection(population.population, self.params["tournament_size"])
          children = generate_offspring(parents,
                                        pc_pixels = self.params["pc_pixels"],
                                        pm_pixels = self.params["pm_pixels"],
                                        zero_prob = self.params["zero_probability"],
                                        m_eta = self.params["m_eta"],
                                        c_eta = self.params["c_eta"])
          offsprings = Population(children, loss_function)
          fe += len(offsprings.population)
          # eval_off_start = time.time()
          offsprings.evaluate()
          # print("eval off:",time.time()-eval_off_start)
          population.population.extend(offsprings.population)
          population.fronts = fast_nondominated_sort(population.population)
          front_num = 0
          new_solutions = []
          while len(new_solutions) + len(population.fronts[front_num]) <= self.params["pop_size"]:
            calculate_crowding_distance(population.fronts[front_num])
            new_solutions.extend(population.fronts[front_num])
            front_num += 1

          calculate_crowding_distance(population.fronts[front_num])
          population.fronts[front_num].sort(key=attrgetter("crowding_distance"), reverse=True)
          new_solutions.extend(population.fronts[front_num][0:self.params["pop_size"] - len(new_solutions)])

          population = Population(new_solutions, loss_function)

        population.fronts = fast_nondominated_sort(population.population)
        self.fitness.append(min(population.population, key=attrgetter('loss')).fitnesses)
        self.completion_procedure(population, loss_function, fe, False)
        print(time.time() - start)
        return

importing Jupyter notebook from operators.ipynb
importing Jupyter notebook from Solutions.ipynb
